In [ ]:
%matplotlib inline
from pprint import pprint

import plot
from simulate import simulate_withdrawals
from harvesting import N_60_RebalanceHarvesting, N_100_RebalanceHarvesting
import harvesting

import itertools
from decimal import Decimal
from montecarlo import conservative
import metrics

In [ ]:
import math
from market import Returns_US_1871
import withdrawal

START_YEAR = 1969

def Pye(p, h):
    vpw = withdrawal.VPW(p, h, years_left=50)
    vpw.stock_growth_rate = Decimal('.08')
    vpw.bond_growth_rate = Decimal('.08')
    return vpw

def compare_em_vs_vpw(series, years=40, title=''):
    (r1, r2) = itertools.tee(series)
    portfolio = (600000, 400000)
    x = simulate_withdrawals(r1, years=years, harvesting=N_60_RebalanceHarvesting, withdraw=Pye, portfolio=portfolio)
    y = simulate_withdrawals(r2, years=years, harvesting=N_60_RebalanceHarvesting, withdraw=withdrawal.VPW, portfolio=portfolio)

    plot.plot_n({'Pye' : [n.withdraw_r for n in x], 'VPW' : [n.withdraw_r for n in y]}, 'Year of Retirement', title)
    plot.plot_n({'Pye' : [n.portfolio_pre.value_r for n in x], 'VPW' : [n.portfolio_pre.value_r for n in y]}, 'Year of Retirement', title)
    
    import pprint
    diffs = [a.withdraw_r - b.withdraw_r for (a,b) in zip(x,y)]
    print(metrics.mean(diffs[11:]))
    pprint.pprint([a.withdraw_r for a in x])
    
def ern_vs_vpw(year, years=40):
    compare_em_vs_vpw(Returns_US_1871().iter_from(year), title='%d: Pye vs VPW' % year, years=years)

ern_vs_vpw(START_YEAR, years=20)